<a href="https://colab.research.google.com/github/shreyasat27/pennylane-27524/blob/main/Building_molecular_Hamiltonians.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial,

you will learn how to use PennyLane to build a representation of the electronic Hamiltonian that can be used to perform quantum simulations of molecules. First, we show how to define the structure of the molecule in terms of the symbols and the coordinates of the atoms. Next, we describe how to solve the Hartree-Fock equations for the target molecule. Finally, we discuss some advanced features that can be used to simulate more complicated systems.

Let’s get started!



Defining the molecular structure

The structure of a molecule is defined by the symbols and the nuclear coordinates of its constituent atoms. It can be specified using different chemical file formats. Within PennyLane, the molecular structure is defined by providing a list with the atomic symbols and a one-dimensional array with the nuclear coordinates in atomic units.

References


Chemical file format
https://en.wikipedia.org/wiki/Chemical_file_format

Atomic units
https://en.wikipedia.org/wiki/Atomic_units

In [7]:
!pip install pennylane

In [8]:
import numpy as np

symbols=["H", "O", "H"]
coordinates = np.array ([-0.0399, -0.0038, 0.0, 1.5780, 0.8540, 0.0, 2.7909, -0.5159, 0.0])

The read_structure() function can also be used to read the molecular geometry from an external file.

In [9]:
#from pennylane import qchem

#symbols, coordinates = qchem.read_structure("h2o.xyz")

**Solving the Hartree-Fock equations**


The molecule’s electronic Hamiltonian is commonly represented using the second-quantization formalism, which we will explore in more detail in the next section. To that aim, a basis of single-particle states needs to be chosen. In quantum chemistry these states are the molecular orbitals which describe the wave function of a single electron in the molecule.

Molecular orbitals are typically represented as a linear combination of atomic orbitals. The expansion coefficients in the atomic basis are calculated using the Hartree-Fock (HF) method. In the HF approximation, each electron in the molecule is treated as an independent particle that moves under the influence of the Coulomb potential due to the nuclei, and a mean field generated by all other electrons. The optimized coefficients are precisely what we need to build the second-quantized Hamiltonian.

PennyLane provides a differentiable Hartree-Fock solver and the functionality to construct a fully-differentiable molecular Hamiltonian.

**Building the Hamiltonian**

In PennyLane we have the molecular_hamiltonian() function which encapsulates all the steps explained above. It simplifies the process of building the electronic Hamiltonian to a single line of code. We just need to input the symbols and the nuclear coordinates of the molecule, as shown below:

In [10]:
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits: {:}".format(qubits))
print("Qubit Hamiltonian")
print(H)

Number of qubits: 14
Qubit Hamiltonian
-46.46418145372084 * I(0) + 12.41262577137833 * Z(0) + -0.12507007495317848 * (Y(0) @ Z(1) @ Y(2)) + -0.12507007495317848 * (X(0) @ Z(1) @ X(2)) + 6.26455005967796e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 6.26455005967796e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.042745029155174454 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.042745029155174454 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.07164205833936418 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10)) + -0.07164205833936418 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10)) + 0.00027347855269445565 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Z(11) @ Y(12)) + 0.00027347855269445565 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Z(11) @ X(12)) + 1.6538777936362172 * Z(2) + 0.23671146232606843 * (Z(0) @ Z(2)) + 7.60426349319744e-05 * (Y(2) @ Z

**Advanced features**


The molecular_hamiltonian() function allows us to define additional keyword arguments to solve the Hartree-Fock equations of more complicated systems. The net charge of the molecule may be specified to simulate positively or negatively charged molecules. For a neutral system we choose

In [11]:
charge =0

We can also specify the spin multiplicity. For the water molecule, which contains ten electrons, the Slater determinant resulting from occupying the five lowest-energy orbitals with two paired electrons per orbital has spin multiplicity one. Alternatively, if we define an occupation where the first four orbitals are doubly occupied and the next two are singly occupied by unpaired electrons, the HF state will have multiplicity three.

For the neutral water molecule we have,

In [12]:
multiplicity = 1

As mentioned above, molecular orbitals are represented as a linear combination of atomic orbitals which are typically modeled as Gaussian-type orbitals. We can specify different types of Gaussian atomic bases. In this example we choose a minimal basis set.

In [13]:
basis_set = 'sto-3g'

PennyLane also allows us to define an active space to perform quantum simulations with a reduced number of qubits. This is done by classifying the molecular orbitals as core, active, and external orbitals:

Core orbitals are always occupied by two electrons.

Active orbitals can be occupied by zero, one, or two electrons.

The external orbitals are never occupied.

Within this approximation, a certain number of active electrons are allowed to populate a finite set of active orbitals.

For the water molecule in a minimal basis set we have a total of ten electrons and seven molecular orbitals. In this example we define an symmetric active space with four electrons and four active orbitals using the active_space() function:

In [14]:
electrons = 10
orbitals = 7
core, active = qchem.active_space(electrons, orbitals, active_electrons =4, active_orbitals = 4)



viewing the results:

In [15]:
print("List of core orbitals: {:}".format(core))
print("List of active orbitals: {:}".format(active))
print("Number of qubits: {:}".format(2 * len(active)))

List of core orbitals: [0, 1, 2]
List of active orbitals: [3, 4, 5, 6]
Number of qubits: 8


Finally, we use the molecular_hamiltonian() function to build the resulting Hamiltonian of the water molecule:

In [16]:
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=charge,
    mult=multiplicity,
    basis=basis_set,
    active_electrons=4,
    active_orbitals=4,
)

print("Number of qubits required to perform quantum simulations: {:}".format(qubits))
print("Hamiltonian of the water molecule")
print(H)



Number of qubits required to perform quantum simulations: 8
Hamiltonian of the water molecule
-73.13888998350859 * I(0) + 0.22757159034432367 * Z(0) + -0.04376228350772 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + -0.04376228350772 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 9.98771733651186e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 9.98771733651186e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.17419424119098043 * Z(2) + 0.16756088655628065 * (Z(0) @ Z(2)) + -0.15958810663095946 * Z(4) + 0.1195206988968364 * (Z(0) @ Z(4)) + -0.00013759633234727283 * (Y(4) @ Z(5) @ Y(6)) + -0.00013759633234727283 * (X(4) @ Z(5) @ X(6)) + -3.626908172703856e-05 * (Z(0) @ Y(4) @ Z(5) @ Y(6)) + -3.626908172703856e-05 * (Z(0) @ X(4) @ Z(5) @ X(6)) + -0.18062549257290825 * Z(6) + 0.1340102991282929 * (Z(0) @ Z(6)) + 0.22757159034432353 * Z(1) + 0.19391187326739356 * (Z(0) @ Z(1)) + -0.03079106522292483 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + -0.03079106522292483 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 7.373

**OpenFermion-PySCF backend**

The molecular_hamiltonian() function can also be used to construct the molecular Hamiltonian with a non-differentiable backend that uses the OpenFermion-PySCF plugin interfaced with the electronic structure package PySCF. This backend can be selected by setting method='pyscf' in molecular_hamiltonian():

In [18]:
!pip install pyscf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 MB 13.4 MB/s eta 0:00:00


In [19]:
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates, method="pyscf")
print(H)

-46.46418132900203 * I(0) + 12.412625796786337 * Z(0) + 0.1250700502026278 * (Y(0) @ Z(1) @ Y(2)) + 0.1250700502026278 * (X(0) @ Z(1) @ X(2)) + -6.264577773736353e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + -6.264577773736353e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.042745032894291324 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.042745032894291324 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.07164199409273314 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10)) + 0.07164199409273314 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10)) + 0.0002734799108736431 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Z(11) @ Y(12)) + 0.0002734799108736431 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Z(11) @ X(12)) + 1.6538779001711001 * Z(2) + 0.2367114864058636 * (Z(0) @ Z(2)) + 7.604546233261617e-05 * (Y(2) @ Z(3) @ Y(4)) + 7.604546233261617e-05 * (X(2

This backend requires the OpenFermion-PySCF plugin to be installed by the user with

In [20]:
!pip install openfermionpyscf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=09335254c19931688f425c44409edc56197909773b025008227a0edbb4e3e5ea
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


Additionally, if you have built your electronic Hamiltonian independently using OpenFermion tools, it can be readily converted to a PennyLane observable using the import_operator() function.

You have completed the tutorial! Now, select your favorite molecule and build its electronic Hamiltonian. To see how simple it is to implement the VQE algorithm to compute the ground-state energy of your molecule using PennyLane, take a look at the tutorial A brief overview of VQE.

